In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
path = '/content/drive/My Drive/PMEmo Dataset/'
!ls /content/drive/My\ Drive/PMEmo\ Dataset/

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
PMEmo2019


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
xd = pd.read_csv(path+'PMEmo2019/features/dynamic_features.csv')
xs = pd.read_csv(path+'PMEmo2019/features/static_features.csv')
yd = pd.read_csv(path+'PMEmo2019/annotations/dynamic_annotations.csv')
ys = pd.read_csv(path+'PMEmo2019/annotations/static_annotations.csv')

In [ ]:
static_data = pd.merge(xs,ys,on='musicId')
static_data.info()
static_data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 767 entries, 0 to 766
Columns: 6376 entries, musicId to Valence(mean)
dtypes: float64(6375), int64(1)
memory usage: 37.3 MB


,musicId,audspec_lengthL1norm_sma_range,audspec_lengthL1norm_sma_maxPos,audspec_lengthL1norm_sma_minPos,audspec_lengthL1norm_sma_quartile1,audspec_lengthL1norm_sma_quartile2,audspec_lengthL1norm_sma_quartile3,audspec_lengthL1norm_sma_iqr1-2,audspec_lengthL1norm_sma_iqr2-3,audspec_lengthL1norm_sma_iqr1-3,audspec_lengthL1norm_sma_percentile1.0,audspec_lengthL1norm_sma_percentile99.0,audspec_lengthL1norm_sma_pctlrange0-1,audspec_lengthL1norm_sma_stddev,audspec_lengthL1norm_sma_skewness,audspec_lengthL1norm_sma_kurtosis,audspec_lengthL1norm_sma_meanSegLen,audspec_lengthL1norm_sma_maxSegLen,audspec_lengthL1norm_sma_minSegLen,audspec_lengthL1norm_sma_segLenStddev,audspec_lengthL1norm_sma_upleveltime25,audspec_lengthL1norm_sma_upleveltime50,audspec_lengthL1norm_sma_upleveltime75,audspec_lengthL1norm_sma_upleveltime90,audspec_lengthL1norm_sma_risetime,audspec_lengthL1norm_sma_leftctime,audspec_lengthL1norm_sma_lpgain,audspec_lengthL1norm_sma_lpc0,audspec_lengthL1norm_sma_lpc1,audspec_lengthL1norm_sma_lpc2,audspec_lengthL1norm_sma_lpc3,audspec_lengthL1norm_sma_lpc4,audspecRasta_lengthL1norm_sma_range,audspecRasta_lengthL1norm_sma_maxPos,audspecRasta_lengthL1norm_sma_minPos,audspecRasta_lengthL1norm_sma_quartile1,audspecRasta_lengthL1norm_sma_quartile2,audspecRasta_lengthL1norm_sma_quartile3,audspecRasta_lengthL1norm_sma_iqr1-2,audspecRasta_lengthL1norm_sma_iqr2-3,...,mfcc_sma_de[12]_peakMeanAbs,mfcc_sma_de[12]_peakMeanMeanDist,mfcc_sma_de[12]_peakMeanRel,mfcc_sma_de[12]_minRangeRel,mfcc_sma_de[12]_meanRisingSlope,mfcc_sma_de[12]_stddevRisingSlope,mfcc_sma_de[12]_meanFallingSlope,mfcc_sma_de[12]_stddevFallingSlope,mfcc_sma_de[13]_flatness,mfcc_sma_de[13]_posamean,mfcc_sma_de[13]_rqmean,mfcc_sma_de[13]_meanPeakDist,mfcc_sma_de[13]_peakDistStddev,mfcc_sma_de[13]_peakRangeAbs,mfcc_sma_de[13]_peakRangeRel,mfcc_sma_de[13]_peakMeanAbs,mfcc_sma_de[13]_peakMeanMeanDist,mfcc_sma_de[13]_peakMeanRel,mfcc_sma_de[13]_minRangeRel,mfcc_sma_de[13]_meanRisingSlope,mfcc_sma_de[13]_stddevRisingSlope,mfcc_sma_de[13]_meanFallingSlope,mfcc_sma_de[13]_stddevFallingSlope,mfcc_sma_de[14]_flatness,mfcc_sma_de[14]_posamean,mfcc_sma_de[14]_rqmean,mfcc_sma_de[14]_meanPeakDist,mfcc_sma_de[14]_peakDistStddev,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope,Arousal(mean),Valence(mean)
0,1,7.318236,0.716432,0.0,2.245124,3.145057,3.956726,0.899933,0.811669,1.711601,0.929235,6.625757,5.696522,1.201596,0.364819,3.121323,0.660851,1.85,0.31,0.404164,0.847261,0.333020,0.027543,0.011268,0.450219,0.514250,0.041762,-1.983495,1.234478,0.090131,-0.537227,0.200833,4.759681,0.007512,0.452895,0.851731,1.055520,1.310368,0.203789,0.254848,...,3.347777,3.361259,-248.3129,0.630844,136.6264,72.24846,130.9828,71.69621,0.641416,1.911776,2.547246,0.115455,0.067832,10.313080,0.522352,3.117261,3.111755,566.1714,0.567876,136.7822,75.50486,126.8567,74.64148,0.642098,1.724395,2.204436,0.121992,0.071012,10.694290,0.552123,2.884782,2.886936,-1339.0630,0.554930,113.55350,57.37954,111.61330,59.35901,0.4000,0.5750
1,4,6.558082,0.703399,0.0,1.606873,2.116417,2.917487,0.509543,0.801071,1.310614,0.311745,5.881815,5.570071,1.088543,1.041186,4.251553,0.422121,1.39,0.27,0.218643,0.733453,0.178175,0.032200,0.010018,0.445956,0.524526,0.032234,-2.041644,1.229984,0.217129,-0.623459,0.225951,50.246180,0.003220,0.901252,0.779240,1.051655,1.419134,0.272415,0.367479,...,3.083433,3.080873,1204.3570,0.594460,115.3460,65.94817,124.0453,59.58847,0.635191,1.747938,2.204957,0.109643,0.061818,8.832486,0.486491,2.705443,2.701887,760.8471,0.733748,111.2992,56.45495,118.3074,61.00229,0.641903,1.651459,2.116423,0.119657,0.067917,8.273178,0.535849,2.659783,2.660821,-2561.7130,0.665665,104.48770,63.52283,106.18320,54.71746,0.2625,0.2875
2,5,8.152512,0.368032,0.0,1.404577,

In [ ]:
dynamic_data = pd.merge(xd,yd,on=['musicId','frameTime'])
dynamic_data.info()
dynamic_data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36434 entries, 0 to 36433
Columns: 264 entries, musicId to Valence(mean)
dtypes: float64(263), int64(1)
memory usage: 73.7 MB


,musicId,frameTime,F0final_sma_mean,voicingFinalUnclipped_sma_mean,jitterLocal_sma_mean,jitterDDP_sma_mean,shimmerLocal_sma_mean,logHNR_sma_mean,audspec_lengthL1norm_sma_mean,audspecRasta_lengthL1norm_sma_mean,pcm_RMSenergy_sma_mean,pcm_zcr_sma_mean,audSpec_Rfilt_sma[0]_mean,audSpec_Rfilt_sma[1]_mean,audSpec_Rfilt_sma[2]_mean,audSpec_Rfilt_sma[3]_mean,audSpec_Rfilt_sma[4]_mean,audSpec_Rfilt_sma[5]_mean,audSpec_Rfilt_sma[6]_mean,audSpec_Rfilt_sma[7]_mean,audSpec_Rfilt_sma[8]_mean,audSpec_Rfilt_sma[9]_mean,audSpec_Rfilt_sma[10]_mean,audSpec_Rfilt_sma[11]_mean,audSpec_Rfilt_sma[12]_mean,audSpec_Rfilt_sma[13]_mean,audSpec_Rfilt_sma[14]_mean,audSpec_Rfilt_sma[15]_mean,audSpec_Rfilt_sma[16]_mean,audSpec_Rfilt_sma[17]_mean,audSpec_Rfilt_sma[18]_mean,audSpec_Rfilt_sma[19]_mean,audSpec_Rfilt_sma[20]_mean,audSpec_Rfilt_sma[21]_mean,audSpec_Rfilt_sma[22]_mean,audSpec_Rfilt_sma[23]_mean,audSpec_Rfilt_sma[24]_mean,audSpec_Rfilt_sma[25]_mean,pcm_fftMag_fband250-650_sma_mean,pcm_fftMag_fband1000-4000_sma_mean,...,audSpec_Rfilt_sma_de[17]_std,audSpec_Rfilt_sma_de[18]_std,audSpec_Rfilt_sma_de[19]_std,audSpec_Rfilt_sma_de[20]_std,audSpec_Rfilt_sma_de[21]_std,audSpec_Rfilt_sma_de[22]_std,audSpec_Rfilt_sma_de[23]_std,audSpec_Rfilt_sma_de[24]_std,audSpec_Rfilt_sma_de[25]_std,pcm_fftMag_fband250-650_sma_de_std,pcm_fftMag_fband1000-4000_sma_de_std,pcm_fftMag_spectralRollOff25.0_sma_de_std,pcm_fftMag_spectralRollOff50.0_sma_de_std,pcm_fftMag_spectralRollOff75.0_sma_de_std,pcm_fftMag_spectralRollOff90.0_sma_de_std,pcm_fftMag_spectralFlux_sma_de_std,pcm_fftMag_spectralCentroid_sma_de_std,pcm_fftMag_spectralEntropy_sma_de_std,pcm_fftMag_spectralVariance_sma_de_std,pcm_fftMag_spectralSkewness_sma_de_std,pcm_fftMag_spectralKurtosis_sma_de_std,pcm_fftMag_spectralSlope_sma_de_std,pcm_fftMag_psySharpness_sma_de_std,pcm_fftMag_spectralHarmonicity_sma_de_std,pcm_fftMag_mfcc_sma_de[1]_std,pcm_fftMag_mfcc_sma_de[2]_std,pcm_fftMag_mfcc_sma_de[3]_std,pcm_fftMag_mfcc_sma_de[4]_std,pcm_fftMag_mfcc_sma_de[5]_std,pcm_fftMag_mfcc_sma_de[6]_std,pcm_fftMag_mfcc_sma_de[7]_std,pcm_fftMag_mfcc_sma_de[8]_std,pcm_fftMag_mfcc_sma_de[9]_std,pcm_fftMag_mfcc_sma_de[10]_std,pcm_fftMag_mfcc_sma_de[11]_std,pcm_fftMag_mfcc_sma_de[12]_std,pcm_fftMag_mfcc_sma_de[13]_std,pcm_fftMag_mfcc_sma_de[14]_std,Arousal(mean),Valence(mean)
0,1,15.5,93.903882,0.693862,0.021792,0.031370,0.036244,-54.431589,3.448038,1.155336,0.180737,0.096815,0.193270,0.302045,0.463922,0.590518,0.738612,0.841255,0.819176,0.921536,1.090391,1.103228,1.096543,1.123980,1.301335,1.553387,1.594355,1.651450,1.636781,1.624078,1.834766,1.851205,1.776009,1.626734,1.357231,1.096293,0.950052,0.900578,1.687298,1.578479,...,0.180320,0.241642,0.202353,0.180725,0.201644,0.171848,0.143129,0.138363,0.189071,0.636440,0.458302,97.709713,199.554137,490.540472,549.636242,0.278175,215.079673,0.283152,8.656297e+05,0.804029,16.831455,0.372496,0.145643,0.739516,1.980103,2.354329,2.285487,3.452269,3.755339,3.466142,3.946592,3.937621,3.406339,3.439967,2.796637,2.787515,2.933549,2.639395,0.437663,0.559975
1,1,16.0,74.883563,0.659456,0.031344,0.032647,0.059305,-63.578455,3.047558,1.080958,0.172719,0.100270,0.229766,0.314629,0.444188,0.589461,0.702757,0.809016,0.810526,0.860255,0.895810,0.989920,1.141196,1.283009,1.391480,1.356735,1.401969,1.490028,1.472794,1.539865,1.731359,1.621135,1.405512,1.423112,1.236362,1.062247,0.980743,0.921031,1.616658,1.284050,...,0.220166,0.239268,0.229283,0.187239,0.224143,0.191088,0.160362,0.161356,0.199835,0.632279,0.589757,276.720620,301.455372,445.209924,593.226026,0.286114,277.429493,0.283511,9.328551e+05,1.907974,92.316307,0.357061,0.191547,0.774155,2.989479,3.188893,1.837241,3.051340,3.197435,4.220361,4.503875,4.643665,3.531094,3.676481,3.779022,2.590293,2.574108,2.742835,0.437775,0.560650
2,1,16.5,96.593653,0.644571,0.031822,0.032635,0.083449,-53.322088,3.236864,1.145600,0.215291,0.074617,0.241898,0.299165,0.407345,0.533610,0.733761,0.843278,1.090084,1.122119,0.996649,1.175123,1.351112,1.355195,1.342002,1.3213

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, Ridge, SGDRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_validate, KFold, train_test_split
from sklearn.metrics import mean_squared_error, make_scorer
# from sklearn.utils import shuffle
# from scipy.stats import pearsonr
# from tqdm import tqdm_notebook
# import IPython.display as ipd

In [ ]:
scorer = ['neg_root_mean_squared_error','neg_mean_absolute_error','neg_median_absolute_error']

# Static Data

In [ ]:
models = {
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "ElasticNet": ElasticNet(),
    "KNN": KNeighborsRegressor(n_neighbors=5),
    "DT": DecisionTreeRegressor(max_depth=5),
    "RF": RandomForestRegressor(max_depth=5, n_estimators=10, max_features=1),
    "AdaBoost": AdaBoostRegressor(n_estimators=10),
    "GBoost": GradientBoostingRegressor(n_estimators=10),
    "MLP": MLPRegressor(hidden_layer_sizes=(1500,200,40), max_iter=2000),
    "SVRrbf": SVR(kernel='rbf', gamma='scale'),
    "SVRpoly": SVR(kernel='poly', gamma='scale'),
    "SVRlinear": SVR(kernel='linear', gamma='scale'),
}

In [ ]:
# Test Set of 77 songs (10%)
train_data, test_data = train_test_split(static_data[static_data.columns[1:]],test_size=0.1,random_state=0)
print(train_data.shape, test_data.shape)

(690, 6375) (77, 6375)


In [ ]:
X = StandardScaler().fit_transform(train_data[train_data.columns[:-2]])
y1 = train_data[train_data.columns[-2]]
y2 = train_data[train_data.columns[-1]]
print(X.shape, y1.shape, y2.shape)

(690, 6373) (690,) (690,)


In [ ]:
print("Training for Arousal")
scoring = {}
for model in models:
    scores = {}
    results = cross_validate(estimator=models[model], X=X, y=y1, cv=10, scoring=scorer, return_train_score=True, n_jobs=-1)
    scores['rmse'] = (-results['train_neg_root_mean_squared_error'].mean(),-results['test_neg_root_mean_squared_error'].mean())
    scores['mean_ae'] = (-results['train_neg_mean_absolute_error'].mean(),-results['test_neg_mean_absolute_error'].mean())
    scores['median_ae'] = (-results['train_neg_median_absolute_error'].mean(),-results['test_neg_median_absolute_error'].mean())
    print(model)
    scoring[model] = scores

Training for Arousal
MLP1
MLP2
MLP3
MLP4


In [ ]:
scores = ['rmse','mean_ae','median_ae']
vals = []
for score in scores:
    values = []
    for model in models:
        values.append(scoring[model][score][1])
    vals.append(values)
table = pd.DataFrame(data=vals,columns=models.keys(),index=scores)
# table.to_csv('Audio_static_arousal_scores.csv')
table.head()

,MLP1,MLP2,MLP3,MLP4
rmse,0.302417,0.262385,0.246938,0.261418
mean_ae,0.234621,0.202662,0.189485,0.204926
median_ae,0.184779,0.159747,0.145517,0.175928


In [ ]:
print("Training for Valence")
scoring = {}
for model in models:
    scores = {}
    results = cross_validate(estimator=models[model], X=X, y=y2, cv=10, scoring=scorer, return_train_score=True, n_jobs=-1)
    scores['rmse'] = (-results['train_neg_root_mean_squared_error'].mean(),-results['test_neg_root_mean_squared_error'].mean())
    scores['mean_ae'] = (-results['train_neg_mean_absolute_error'].mean(),-results['test_neg_mean_absolute_error'].mean())
    scores['median_ae'] = (-results['train_neg_median_absolute_error'].mean(),-results['test_neg_median_absolute_error'].mean())
    print(model)
    scoring[model] = scores

In [ ]:
scores = ['rmse','mean_ae','median_ae']
vals = []
for score in scores:
    values = []
    for model in models:
        values.append(scoring[model][score][1])
    vals.append(values)
table = pd.DataFrame(data=vals,columns=models.keys(),index=scores)
# table.to_csv('Audio_static_valence_scores.csv')
table.head()

# Dynamic Data

In [ ]:
train_songs, test_songs = train_test_split(pd.unique(dynamic_data[dynamic_data.columns[0]]),test_size=0.1,random_state=0)
print(train_songs.shape,test_songs.shape)
train_data = dynamic_data[[dynamic_data.musicId[i] in train_songs for i in range(dynamic_data.shape[0])]]
test_data = dynamic_data[[dynamic_data.musicId[i] in test_songs for i in range(dynamic_data.shape[0])]]
print(train_data.shape,test_data.shape)

(690,) (77,)
(32896, 264) (3538, 264)


In [ ]:
X = StandardScaler().fit_transform(train_data[train_data.columns[2:-2]]) 
y1 = train_data[train_data.columns[-2]]
y2 = train_data[train_data.columns[-1]]
print(X.shape, y1.shape, y2.shape)

(32896, 260) (32896,) (32896,)


In [ ]:
models = {
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "ElasticNet": ElasticNet(),
    "KNN": KNeighborsRegressor(n_neighbors=10, algorithm='kd_tree'),
    "DT": DecisionTreeRegressor(max_depth=8),
    "RF": RandomForestRegressor(max_depth=8, n_estimators=10, max_features=1),
    "AdaBoost": AdaBoostRegressor(n_estimators=10),
    "GBoost": GradientBoostingRegressor(n_estimators=10),
    "MLP": MLPRegressor(hidden_layer_sizes=(200,50), max_iter=2000, learning_rate_init=0.01),
    "LinearSVR": LinearSVR(),
}

In [ ]:
print("Training for Arousal")
scoring = {}
for model in models:
    scores = {}
    results = cross_validate(estimator=models[model], X=X, y=y1, groups=train_data.musicId, cv=5, scoring=scorer, return_train_score=True, n_jobs=-1)
    scores['rmse'] = (-results['train_neg_root_mean_squared_error'].mean(),-results['test_neg_root_mean_squared_error'].mean())
    scores['mean_ae'] = (-results['train_neg_mean_absolute_error'].mean(),-results['test_neg_mean_absolute_error'].mean())
    scores['median_ae'] = (-results['train_neg_median_absolute_error'].mean(),-results['test_neg_median_absolute_error'].mean())
    print(model)
    scoring[model] = scores

Training for Arousal
Lasso
Ridge
ElasticNet
KNN
DT
RF
AdaBoost
GBoost
MLP
LinearSVR


In [ ]:
scores = ['rmse','mean_ae','median_ae']
vals = []
for score in scores:
    values = []
    for model in models:
        values.append(scoring[model][score][1])
    vals.append(values)
table = pd.DataFrame(data=vals,columns=models.keys(),index=scores)
table.to_csv('Audio_dynamic_arousal_scores.csv')
table.head()

,Lasso,Ridge,ElasticNet,KNN,DT,RF,AdaBoost,GBoost,MLP,LinearSVR
rmse,0.176279,0.124283,0.176279,0.140355,0.139526,0.139343,0.128878,0.135815,0.136979,0.153605
mean_ae,0.146878,0.096349,0.146878,0.109394,0.108697,0.113478,0.103707,0.110877,0.107565,0.114838
median_ae,0.137409,0.079451,0.137409,0.088764,0.088560,0.101365,0.089220,0.098843,0.089720,0.090898


In [ ]:
print("Training for Valence")
scoring = {}
for model in models:
    scores = {}
    results = cross_validate(estimator=models[model], X=X, y=y2, groups=train_data.musicId, cv=5, scoring=scorer, return_train_score=True, n_jobs=-1)
    scores['rmse'] = (-results['train_neg_root_mean_squared_error'].mean(),-results['test_neg_root_mean_squared_error'].mean())
    scores['mean_ae'] = (-results['train_neg_mean_absolute_error'].mean(),-results['test_neg_mean_absolute_error'].mean())
    scores['median_ae'] = (-results['train_neg_median_absolute_error'].mean(),-results['test_neg_median_absolute_error'].mean())
    print(model)
    scoring[model] = scores

Training for Valence
Lasso
Ridge
ElasticNet
KNN
DT
RF
AdaBoost
GBoost
MLP
LinearSVR


In [ ]:
scores = ['rmse','mean_ae','median_ae']
vals = []
for score in scores:
    values = []
    for model in models:
        values.append(scoring[model][score][1])
    vals.append(values)
table = pd.DataFrame(data=vals,columns=models.keys(),index=scores)
table.to_csv('Audio_dynamic_valence_scores.csv')
table.head()

,Lasso,Ridge,ElasticNet,KNN,DT,RF,AdaBoost,GBoost,MLP,LinearSVR
rmse,0.159240,0.128970,0.159240,0.139971,0.138521,0.132971,0.129730,0.133626,0.138184,0.176479
mean_ae,0.131580,0.100408,0.131580,0.108720,0.108196,0.107973,0.105473,0.108691,0.108191,0.130634
median_ae,0.122699,0.082909,0.122699,0.087486,0.088489,0.095064,0.091745,0.095440,0.088863,0.104563
